## Автоматическое дифференцирование

In [45]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

### Вычисление значения производной в точке

Допустим, нужно вычислить производную функции y = x^2 в точке x=-2

In [2]:
x = tf.Variable(-2.0)

In [3]:
with tf.GradientTape() as tape: #это называется менеджер контекста
    y = x ** 2

In [4]:
df = tape.gradient(y, x)
df

<tf.Tensor: shape=(), dtype=float32, numpy=-4.0>

В tf.GradientTape записываются все промежуточные вычисления. Т.е. создается граф вычислений на основании заданной функции. Метод tape.gradient исполняет обратный проход по графу вычислений и вычисляет значение производной функции y для точки x

### Вычисление производной функции от векторов параметров

Функция:  y = x * w + b,  где w и b - векторы параметров. 

На основании функции y определим функцию loss = (1/N) * сумма (yi^2)

Для функции loss будем рассчитывать значения производных по w и b

#### Определяем значения параметров

In [5]:
w = tf.Variable(tf.random.normal((3, 2))) #в этой точке будет рассчитываться производная
b = tf.Variable(tf.zeros(2, dtype=tf.float32)) #и в этой точке будет рассчитываться производная
x = tf.Variable([[-2.0, 1.0, 3.0]]) #матрица (1, 3)
print(w.numpy(), b.numpy(), x.numpy(), sep='\n\n')

[[ 0.16315922  0.3749139 ]
 [-0.2832861   0.328028  ]
 [-0.6768528  -0.91743714]]

[0. 0.]

[[-2.  1.  3.]]


#### Определяем функцию loss

In [6]:
with tf.GradientTape() as tape:
    y = x @ w + b # матричное умножение x w + смещение b
    loss = tf.reduce_mean(y ** 2) #среднее арифметическое квадратов yi-ых

#### Вычисление производных по параметру w и b

In [7]:
df = tape.gradient(loss, [w, b])
print (df[0], df[1], sep='\n\n')

tf.Tensor(
[[ 5.2803264  6.3482227]
 [-2.6401632 -3.1741114]
 [-7.9204893 -9.522334 ]], shape=(3, 2), dtype=float32)

tf.Tensor([-2.6401632 -3.1741114], shape=(2,), dtype=float32)


Далее эти значения могут быть использованы, например, для нахождения минимума функции loss

#### Важно! Функция должна зависеть от параметров типа tf.Variable!

#### Для параметров типа tf.constant производная не будет рассчитываться (будет None)

Вот пример

In [8]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.constant(1.5)
with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b # посчитаем производную для f в точка x и b

In [9]:
df = tape.gradient(f, [x, b])
print (df[0], df[1], sep='\n\n')

tf.Tensor(3.0, shape=(), dtype=float32)

None


GradientTape не производит вычислений для объектов типа константа

#### Запрет на вычисление производных по переменным (trainable=False)

In [10]:
x = tf.Variable(0, dtype=tf.float32, trainable=False)
b = tf.Variable(1.5)
with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b

In [11]:
df = tape.gradient(f, [x, b])
print (df[0], df[1], sep='\n\n')

None

tf.Tensor(5.0, shape=(), dtype=float32)


Таким образом, мы не вносим данные по x в объект tape

#### При сложении тензоров-переменных с константами, они превращаются в константы

In [12]:
x = tf.Variable(0, dtype=tf.float32, trainable=False)
b = tf.Variable(1.5) + 1.0 # прибавим число
with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b

In [13]:
df = tape.gradient(f, [x, b])
print (df[0], df[1], sep='\n\n')

None

None


#### Отслеживание переменных

При реализации GradientTape можно задавать, какие переменные нужно отслеживать. В обучении нейросетей участвует множество переменных, поэтому это занимает много времени и ресурсов. Важно задавать, какие переменные нужны для расчетов, а каие нет

In [14]:
# tf.GradientTape(watch_accessed_variables=False) отключает отслеживание всех переменных

In [19]:
x = tf.Variable(2, dtype=tf.float32, trainable=False)
b = tf.Variable(-1.5)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x) #отслеживаем только переменную x
    f = x ** 3 + b * x
df = tape.gradient(f, [x, b])

In [20]:
print (df[0], df[1], sep='\n')

tf.Tensor(10.5, shape=(), dtype=float32)
None


Производная посчиталась только для переменной, которую задали для отслеживания в контекстном менеджере (конструкция with.. as..)

Если мы отслеживаем переменную x, а от нее зависит другая переменная y, и нужно посчитать производную функции f по y, то tensorflow включит y в отслеживание и посчитает производную по  y

In [21]:
x = tf.Variable(1, dtype=tf.float32, trainable=False)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
df = tape.gradient(f, y)

In [23]:
print(df)

tf.Tensor(4.0, shape=(), dtype=float32)


#### Объект tf.GradientTape нельзя просто так вызывать подряд

In [24]:
x = tf.Variable(1, dtype=tf.float32, trainable=False)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
df = tape.gradient(f, y)
df = tape.gradient(f, x)

RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)

Дело в том, что когда в первый раз вызывается метод tape.gradient, он высвобождает все ресурсы, и второй раз его нельзя вызвать. Поэтому, если надо посчитать производные по двум переменным, есть 2 различных способа:

1 способ - сразу посчитать обе производные

In [28]:
x = tf.Variable(1, dtype=tf.float32, trainable=False)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
df = tape.gradient(f, [y, x])
print(df[0], df[1], sep='\n')

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


2 способ - задать флаг persistent=True

In [30]:
x = tf.Variable(1, dtype=tf.float32, trainable=False)
with tf.GradientTape(watch_accessed_variables=False, persistent=True) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
df_y = tape.gradient(f, y)

df_x = tape.gradient(f, x)
print(df_y, df_x, sep='\n')

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


Но при этом, удаление ресурса лежит на плечах программиста. После выполнения расчета градиентов, необходимо освободить память, удали объект tape

In [31]:
del tape

### Определение градиента для векторных величин

#### Допустим, надо найти значение градиента для скалярной функции x, выходной функцией является векторная функция y = (y1, y2). Результирующий градиент будет скаляром

<img src='tf3 pic1.png'>

In [32]:
x = tf.Variable(1.0)
with tf.GradientTape() as tape:
    y = [2.0, 3.0] * x ** 2
df = tape.gradient(y, x)
df

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

Получается, считается градиент в точках y1 и y2, затем они складываются

#### Другой случай - допустим, надо найти значение градиента для вектора (x1, x2), выходной функцией которого - скаляр y. Значения градиента (производной) будут вектором

<img src='tf3 pic2.png'>

In [35]:
x = tf.Variable([1.0, 2.0])
with tf.GradientTape() as tape:
    y = tf.reduce_sum([2.0, 3.0] * x ** 2)
df = tape.gradient(y, x)
print(df)

tf.Tensor([ 4. 12.], shape=(2,), dtype=float32)


#### Условия внутри контекстного менеджера

In [38]:
x = tf.Variable(1.0)
with tf.GradientTape() as tape:
    if x < 2.0:
        y = x ** 2
    else:
        y = x ** 3
df = tape.gradient(y, x)
print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


### Частые ошибки

#### Задание функции вне контекстного менеджера

In [39]:
x = tf.Variable(1.0)
y = 2 * x + 1 # функция задана вне контекстного менеджера
with tf.GradientTape() as tape:
    z = y ** 2
df = tape.gradient(z, x)
print(df)

None


Здесь функция y, хоть и зависит от x, но она задана не в менеджере контекста, поэтому она не отслеживается. Поэтому градиент не считается. Правильно делать так:

In [40]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    y = 2 * x + 1
    z = y ** 2
df = tape.gradient(z, x)
print(df)

tf.Tensor(12.0, shape=(), dtype=float32)


#### Случайная замена переменной на константу

In [42]:
x = tf.Variable(1.0)
for n in range(3):
    with tf.GradientTape() as tape:
        y = x ** 2 + 2 * x
    df = tape.gradient(y, x)
    print(df)
    
    x = x + 1 #так делать не надо!

tf.Tensor(4.0, shape=(), dtype=float32)
None
None


В первую итерацию градиент посчитался, а затем x превратился в константу, т.к. сложился с числом. И на 2-й итерации градиент не посчитался. Правильно вот так:

In [44]:
x = tf.Variable(1.0)
for n in range(3):
    with tf.GradientTape() as tape:
        y = x ** 2 + 2 * x
    df = tape.gradient(y, x)
    print(df)
    
    x.assign_add(1.0)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


#### Использование сторонних библиотек для вычисления функций

In [47]:
x = tf.Variable(1.0)
with tf.GradientTape() as tape:
    y = tf.constant(2.0) + np.square(x) # x возводится в квадрат при помощи numpy! так делать не надо
df = tape.gradient(y, x)
print(df)

None


Надо делать так:

In [48]:
x = tf.Variable(1.0)
with tf.GradientTape() as tape:
    y = tf.constant(2.0) + x ** 2
df = tape.gradient(y, x)
print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


Все вычисления нужно выполнять на уровне tensorflow (чтобы они сохранялись в памяти). На уровень других пакетов переходить нельзя

#### Неправильный тип данных

In [50]:
x = tf.Variable(1) # задан тип integer, а нужен float!
with tf.GradientTape() as tape:
    y = x ** 2
df = tape.gradient(y, x)
print(df)

None


Правильно вот так:

In [51]:
x = tf.Variable(1, dtype=tf.float32) #или 1.0
with tf.GradientTape() as tape:
    y = x ** 2
df = tape.gradient(y, x)
print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


#### Формулы целевой функции в неявном виде

In [52]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)
with tf.GradientTape() as tape:
    w.assign_add(x)
    y = w ** 2 #здесь подразумевается, что y = (w + x) ** 2, w определяется в неявном виде
df = tape.gradient(y, x)
print(df)

None


Правильно сделать так. Задать промежуточную переменную s, которая будет равна сумме w + x

In [53]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)
with tf.GradientTape() as tape:
    s = w + x # это явное задание
    y = s ** 2
df = tape.gradient(y, x)
print(df)

tf.Tensor(6.0, shape=(), dtype=float32)
